In [68]:
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import re
import cx_Oracle

# XML file path
xml_file_path = 'D:\\W\\RELEX\\RELEX\\Broker and Vendor\\Inbound Files\\VENDOR.xml'

# Open the XML file
with open(xml_file_path, 'r', encoding='utf-8') as file:
    xml_data = file.read()

# Database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

# Replace these values with your Oracle database connection details
username = 'apps'
password = 'apps'
host = 'csebsd2db.cswg.com' #EBSD2
port = '1521'
service_name = 'csebsd2_int' #EBSD2

# Create a connection
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)
connection = cx_Oracle.connect(username, password, dsn)

# Table name
table_name = "cspoms.XXPO_ORCL2CSPOMS_VEND_STG".upper()

# Split the XML data into lines
lines = xml_data.splitlines()

# Regular expression pattern to match the elements
pattern = re.compile(r"<([^>]+)>([^<]+)</\1>")
tag_to_column = {
    'BICEPSVENDOR': 'BIC_VENDOR_NUMBER',
    'BIC_VENDOR_FACILITY':'BIC_VENDOR_FACILITY',
    'BUYERNUMBER': 'BIC_BUYER_NUMBER',
    'CRPLEVEL': 'BIC_CRP_LEVEL',
    'STATUS': 'BIC_STATUS',
    'PORECOMMENDEDIND': 'BIC_PO_RECOMMENDED_IND',
    'AFETYPE1': 'BIC_AFE_TYPE1',
    'AFEFAXNUMBER1': 'BIC_AFE_FAX_NUMBER',
    'AFECONTACT1': 'BIC_AFE_CONTACT',
    'MINIMUMQUANTITY': 'BIC_MINIMUM_QUANTITY',
    'MINIMUMTYPE': 'BIC_MINIMUM_TYPE',
    'BKTQUANTITY1': 'BIC_BRACKET_QUANTITY',
    'CURRENTBKTNUMBER': 'BIC_CURRENT_BKT_NUMBER',
    'CURRENTBKTQUANTITY': 'BIC_CURRENT_BKT_QUANTITY',
    'CURRENTBKTTYPE': 'BIC_CURRENT_BKT_TYPE',
    'MAXIMUMQUANTITY': 'BIC_MAXIMUM_QUANTITY',
    'MAXIMUMTYPE': 'BIC_MAXIMUM_TYPE',
    'ORDERINTERVALWEEKS': 'BIC_ORDER_INTERVAL_WEEKS',
    'LEADTIMESTATEDWEEKS': 'BIC_LEAD_TIME_STATED_WEEKS',
    'FIXEDREVIEWDAY1': 'BIC_FIXED_REVIEW_DAY1',
    'FIXEDREVIEWDAY2': 'BIC_FIXED_REVIEW_DAY2',
    'FIXEDREVIEWDAY3': 'BIC_FIXED_REVIEW_DAY3',
    'FIXEDREVIEWDAY4': 'BIC_FIXED_REVIEW_DAY4',
    'FIXEDREVIEWDAY5': 'BIC_FIXED_REVIEW_DAY5',
    'FIXEDREVIEWDAY6': 'BIC_FIXED_REVIEW_DAY6',
    'FIXEDREVIEWDAY7': 'BIC_FIXED_REVIEW_DAY7',
    'TARGETSERVICELEVEL': 'BIC_TARGET_SERVICE_LEVEL',
    'NAME': 'BIC_NAME',
    'ADDRESS1': 'BIC_ADDRESS_1',
    'CITY': 'BIC_CITY',
    'STATE': 'BIC_STATE',
    'ZIP': 'BIC_ZIP',
    'PHONE': 'BIC_PHONE',
    'CONTACT': 'BIC_CONTACT',
    'TRANSFERMATCHTYPE': 'BIC_TRANSFER_MATCH_TYPE',
    'PAYABLEVENDORNBR': 'BIC_PAYABLE_VENDOR_NBR',
    'LEADTIMEWEEKSTYPE': 'BIC_LEAD_TIME_WEEKS_TYPE',
    'DUNSREMITNO': 'BIC_DUNS_REMIT_NO',
    'DUNSREMITSUFFIX': 'BIC_DUNS_REMIT_SUFFIX',
    'FACILITYSHIPTO': 'BIC_FACILITY_SHIP_TO',
    'PRESCHED': 'BIC_PRE_SCHED_FLAG',
    'BROKERNUMBER': 'BIC_BROKER_NUMBER',
    'FREIGHTALLOW': 'BIC_FREIGHT_ALLOW',
    'FREIGHTALLOWTYPE': 'BIC_FREIGHT_ALLOW_TYPE',
    'TERMSPERCENT': 'BIC_TERMS_PERCENT',
    'TERMSDAYS': 'BIC_TERMS_DAYS',
    'TERMSNETDAYS': 'BIC_TERMS_NET_DAYS',
    'TERMSBASE': 'BIC_TERMS_BASE',
    'BILLFORSHORTEDDEALSFLAG': 'BIC_BILL_FOR_SHORT_DEALS_FLAG',
    'TRANSITDAYS': 'BIC_TRANSIT_DAYS',
    'DUNSNO': 'BIC_DUNS_NO',
    '2NDMINQUANTITY': 'BIC_2ND_MIN_QUANTITY',
    '2NDMINTYPE': 'BIC_2ND_MIN_TYPE',
    '2NDMAXQUANTITY': 'BIC_2ND_MAX_QUANTITY',
    '2NDMAXTYPE': 'BIC_2ND_MAX_TYPE',
    'FLAGPREPAID': 'BIC_FLAG_PREPAID',
    'FLAGPREPAYANDADD': 'BIC_FLAG_PREPAY_AND_ADD',
    'FLAGFREIGHTBILL': 'BIC_FLAG_FREIGHT_BILL',
    'FLAGBACKHAUL': 'BIC_FLAG_BACKHAUL',
    'FLAGTRUCK': 'BIC_FLAG_TRUCK',
    'FLAGRAIL': 'BIC_FLAG_RAIL',
    'BACKHAUL': 'BIC_BACKHAUL',
    'BACKHAULTYPE': 'BIC_BACKHAUL_TYPE',
    'FREIGHTBILL': 'BIC_FREIGHTBILL',
    'ORDERSEQUENCE': 'BIC_ORDER_SEQUENCE',
    'SSAALLOWPERCENT': 'BIC_SSA_ALLOW_PERCENT',
    'SSAALLOWBASIS': 'BIC_SSA_ALLOW_BASIS',
    'SSAALLOWACCOUNT': 'BIC_SSA_ALLOW_ACCOUNT',
    'TRANSACTIONID': 'BIC_TRANSACTION_ID',
    'COUNTRY': 'BIC_COUNTRY',
    'ORDERFILTER': 'BIC_FILTER_VENDOR_FLAG',
    'LBLIMIT': 'BIC_LOAD_BLDG_LIMIT_FLAG',
    'WAREHOUSECODE': 'BIC_WAREHOUSE_CODE',
    'FLAGEXTBCKHL': 'BIC_EXTERNAL_BACKHAUL'
}
print('tag_to_column : ',len(tag_to_column))
# Loop through each line in the XML data
for line in lines:
    # Skip empty lines or lines containing only whitespace
    if not line.strip():
        continue

    # Define a dictionary to store values
    element_values = {}

    # Find all matches in the text data
    matches = re.findall(pattern, line)

    # Store the matches in the dictionary
    for tag, value in matches:
        element_values[tag] = value.strip()
    # Assuming 'BICEPSVENDOR' is present in element_values
    if 'BICEPSVENDOR' in element_values:
        bicepsvendor_value = element_values['BICEPSVENDOR']

        # Check if the value has at least 3 characters
        if len(bicepsvendor_value) >= 3:
            # Extract the first 3 digits
            facility_value = bicepsvendor_value[:3]

            # Assign the first 3 digits to a new key
            element_values['BIC_VENDOR_FACILITY'] = facility_value

            # Update 'BICEPSVENDOR' with the remaining value
            element_values['BICEPSVENDOR'] = bicepsvendor_value[3:]
    print('element_values:############',element_values)
    # Now 'BIC_VENDOR_FACILITY' and updated 'BICEPSVENDOR' are present in element_values
    # Replace column names with the ones from tag_to_column
    mapped_values = {tag_to_column[key]: value for key, value in element_values.items() if key in tag_to_column}
    print('mapped_values:########0',mapped_values)
    # Generate SQL INSERT statement
    columns_str = ",".join(mapped_values.keys())
    values_str = ",".join(f"'{v}'" for v in mapped_values.values())

    insert_query = f"INSERT INTO {table_name} ({columns_str}) VALUES ({values_str});"
    print(insert_query)

    # Establish a database connection and execute the INSERT statement
#     with psycopg2.connect(**db_params) as conn:
#         with conn.cursor() as cursor:
#             cursor.execute(insert_query)

    print(f"Record inserted: {mapped_values}")

# Establish a database connection and execute the INSERT statement
with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cursor:
    # Execute the SQL statement
        
        # Execute the SQL statement to select all rows from the table
        cursor.execute("SELECT * FROM XXPO_ORCL2CSPOMS_VEND_STG")

        # Fetch all rows into a list of tuples
        result_rows = cursor.fetchall()

        # Get the column names from the cursor description
        column_names = [desc[0] for desc in cursor.description]

        # Create a DataFrame using the result and column names
        stg_df = pd.DataFrame(result_rows, columns=column_names)

        # Print or use the DataFrame as needed
        print(stg_df)
        # Extract the 'bic_vendor_number' column and convert it to a list
        bic_vendor_numbers_list = stg_df['bic_vendor_number'].tolist()

        # Convert the list of strings to a list of integers
        bic_vendor_numbers_list = list(map(int, bic_vendor_numbers_list))

        # Print or use the list as needed
        print(bic_vendor_numbers_list)


# Print or use the new value as needed
# print("New CS_VENDOR_ID:", new_cs_vendor_id)

# Convert the list of numbers to a string with quotes for the SQL query
vendor_numbers_str = ', '.join([f"'{num}'" for num in bic_vendor_numbers_list])


# Construct the SQL query
sql_query = f"SELECT * FROM APPS.XXAP_VENDOR_MASTER WHERE VENDOR_NBR IN ({vendor_numbers_str})"

oracle_df = pd.read_sql(sql_query, connection)
oracle_df['VENDOR_NBR'] = oracle_df['VENDOR_NBR'].str.zfill(6)
# Display the DataFrame
print(oracle_df.head())

# Merge oracle_df with stg_df based on the common column 'bic_vendor_number' and 'VENDOR_NBR'
merged_df = pd.merge(stg_df, oracle_df, how='left', left_on='bic_vendor_number', right_on='VENDOR_NBR')

# Display the merged DataFrame
print(merged_df)

# XVM_df = merged_df[["VENDOR_SITE_ID", 'VENDOR_NBR', 'CONNECTING_VENDOR', 'BUYER_VENDOR_NAME', 'bic_vendor_facility',
#                     'SHIP_TO_LOCATION_NAME', 'VENDOR_STATUS', 'AP_VENDOR_NBR', 'AP_VENDOR_NAME', 'BICEPS_BUYER_CODE',
#                     'BUYER_NAME', 'bic_warehouse_code', 'VENDOR_FOODLINK_FLG', 'ADDR_LINE1', 'ADDR_LINE2', 'ADDR_LINE3',
#                     'CITY', 'STATE', 'ZIP_CODE', 'COUNTRY', 'PHONE_NBR', 'VENDOR_CONTACT_NAME',
#                     'BILL_TO_LOCATION_NAME', 'VENDOR_BROKER_NBR', 'VENDOR_BROKER_NAME', 'bic_flag_truck',
#                     'bic_flag_rail', 'bic_afe_contact', 'bic_afe_fax_number'
#                     ,'MAIN_FACILITY','bic_freight_allow']]
#last two columns for testing
merged_df = merged_df.fillna('')
# Rename columns
merged_df = merged_df.rename(columns={
    'VENDOR_SITE_ID': 'CS_VEND_SITE_ID',
    'VENDOR_NBR': 'VENDOR_NUMBER',
    'BUYER_VENDOR_NAME': 'VENDOR_NAME',
    'bic_vendor_facility': 'VENDOR_FACILITY',
    'BICEPS_BUYER_CODE': 'BUYER_ID',
    'bic_warehouse_code': 'WAREHOUSE_CODE',
    'VENDOR_FOODLINK_FLG': 'FOOD_LINK_FLAG',
    'VENDOR_BROKER_NBR': 'BROKER_NUMBER',
    'VENDOR_BROKER_NAME': 'BROKER_NAME',
    'bic_flag_truck': 'TRUCK_FLAG',
    'bic_flag_rail': 'RAIL_FLAG',
    'bic_afe_contact': 'PO_TRANSMISSION_TYPE',
    'bic_afe_fax_number': 'PO_TRANSMISSION_VALUE',
    'bic_freight_allow': 'FREIGHT_ALLOW_BUY_RATE'


    
})

# Now XVM_df contains columns with the desired names

# ,'BROKER ADDRESS_1'
# ,'BROKER ADDRESS_2'
# ,'BROKER CITY'
# ,'BROKER STATE'
# ,'BROKER ZIP'
# ,'BROKER COUNTRY'
# Generate the INSERT statement with ON CONFLICT
insert_statement = f"INSERT INTO cspoms.XXPO_VENDOR_MASTER ({', '.join(merged_df.columns)}) VALUES\n"

for index, row in merged_df.iterrows():
    values = ', '.join([repr(val) for val in row.values])
    insert_statement += f"({values}),\n"

# Remove the trailing comma and newline
insert_statement = insert_statement.rstrip(',\n')

# Add ON CONFLICT clause for upsert
conflict_columns = ['VENDOR_NUMBER', 'VENDOR_FACILITY']  # Specify the conflict columns
update_set_clause = ', '.join([f"{col} = EXCLUDED.{col}" for col in merged_df.columns])
insert_statement += f"\nON CONFLICT ({', '.join(conflict_columns)}) DO UPDATE SET {update_set_clause};"
# SHIP_TO_LOCATION_NAME need to insert this to FACILITY_NAME too
print(insert_statement)
# Establish a connection
conn = psycopg2.connect(**db_params)

# Create a cursor
cursor = conn.cursor()

try:
    # Execute the INSERT statement
    cursor.execute(insert_statement)

    # Commit the changes
#     conn.commit()
    print("Data inserted successfully.")
except Exception as e:
    # Rollback in case of an error
    conn.rollback()
    print(f"Error: {e}")
finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()

tag_to_column :  74
element_values:############ {'BICEPSVENDOR': '445932', 'BUYERNUMBER': 'J8', 'CRPLEVEL': '', 'STATUS': 'A', 'PORECOMMENDEDIND': 'B', 'AFETYPE1': '', 'AFEFAXNUMBER1': '', 'AFECONTACT1': '', 'MINIMUMQUANTITY': '0000008', 'MINIMUMTYPE': 'P', 'BKTQUANTITY1': '0000001', 'CURRENTBKTNUMBER': '0', 'CURRENTBKTQUANTITY': '0000026', 'CURRENTBKTTYPE': 'P', 'MAXIMUMQUANTITY': '0000026', 'MAXIMUMTYPE': 'P', 'ORDERINTERVALWEEKS': '000.00', 'LEADTIMESTATEDWEEKS': '002.01', 'FIXEDREVIEWDAY1': '', 'FIXEDREVIEWDAY2': '', 'FIXEDREVIEWDAY3': '', 'FIXEDREVIEWDAY4': '', 'FIXEDREVIEWDAY5': '', 'FIXEDREVIEWDAY6': '', 'FIXEDREVIEWDAY7': '', 'TARGETSERVICELEVEL': '0095', 'NAME': 'INTL. TRADE IMPORTS', 'ADDRESS1': '253 CASTLEBERRY INDUSTRIAL DR', 'CITY': 'CUMMINGS', 'STATE': 'GA', 'ZIP': '30040', 'PHONE': '404 795 0352', 'CONTACT': 'RICARDO SANCHEZ', 'TRANSFERMATCHTYPE': 'I', 'PAYABLEVENDORNBR': '73161', 'LEADTIMEWEEKSTYPE': 'B', 'DUNSREMITNO': '000000000', 'DUNSREMITSUFFIX': '', 'FACILITYSHIPT

   stg_seq_id bic_vendor_facility bic_vendor_number bic_buyer_number  \
0          11                 252            008571               ZZ   
1          12                 905            008571               ZZ   
2          13                 251            445932               J8   
3          14                 252            447709               J8   

  bic_crp_level bic_status bic_po_recommended_ind bic_afe_type1  \
0                        A                      B                 
1                        A                      B                 
2                        A                      B                 
3                        A                      B                 

  bic_afe_fax_number bic_afe_contact  ... attribute7 attribute8 attribute9  \
0                                     ...       None       None       None   
1                                     ...       None       None       None   
2                                     ...       None       None      

Error: syntax error at or near "'2022-02-25 11:02:50'"
LINE 2: ...218150, '008571', 'McELRONE SALES INC', Timestamp('2022-02-2...
                                                             ^



In [53]:
merged_df['CONNECTING_VENDOR']

0    None
1    None
2    None
3    None
Name: CONNECTING_VENDOR, dtype: object

In [ ]:
cursor.execute("SELECT nextval('cs_vendor_id_seq')")
        # Fetch the result into a variable
        new_cs_vendor_id = cursor.fetchone()[0]

In [18]:

for i in merged_df.columns:
    print(i)

stg_seq_id
bic_vendor_facility
bic_vendor_number
bic_buyer_number
bic_crp_level
bic_status
bic_po_recommended_ind
bic_afe_type1
bic_afe_fax_number
bic_afe_contact
bic_minimum_quantity
bic_minimum_quantity_sign
bic_minimum_type
bic_bracket_quantity
bic_bracket_quantity_sign
bic_current_bkt_number
bic_current_bkt_quantity
bic_current_bkt_quantity_sign
bic_current_bkt_type
bic_maximum_quantity
bic_maximum_quantity_sign
bic_maximum_type
bic_order_interval_weeks
bic_order_interval_weeks_sign
bic_lead_time_stated_weeks
bic_lead_time_stated_wks_sign
bic_fixed_review_day1
bic_fixed_review_day2
bic_fixed_review_day3
bic_fixed_review_day4
bic_fixed_review_day5
bic_fixed_review_day6
bic_fixed_review_day7
bic_target_service_level
bic_target_service_level_sign
bic_name
bic_address_1
bic_city
bic_state
bic_zip
bic_phone
bic_contact
bic_transfer_match_type
bic_payable_vendor_nbr
bic_lead_time_weeks_type
bic_duns_remit_no
bic_duns_remit_suffix
bic_facility_ship_to
bic_pre_sched_flag
bic_broker_number


In [28]:
selected_columns = [
    'USE_CYCLE_STOCK_AS_MIN',
    'DAYS_TO_BUILD_TO',
    'USE_CYCLE_STOCK_AS_MIN',
    'USE_FAC_WHSE_SS',
    'MAIN_FACILITY',
    'LEAD_TIME_OVRD_DAYS_1',
    'LEAD_TIME_OVRD_DAYS_2',
    'LEAD_TIME_OVRD_DAYS_3',
    'LEAD_TIME_OVRD_DAYS_4',
    'LEAD_TIME_OVRD_DAYS_5',
    'LEAD_TIME_OVRD_DAYS_6',
    'LEAD_TIME_OVRD_DAYS_7',
    'APPLY_RATIONING',
    'OP_DAYS_1ST_RUN_1',
    'OP_DAYS_1ST_RUN_2',
    'OP_DAYS_1ST_RUN_3',
    'OP_DAYS_1ST_RUN_4'
]

selected_df = merged_df[selected_columns]
selected_df

,USE_CYCLE_STOCK_AS_MIN,DAYS_TO_BUILD_TO,USE_CYCLE_STOCK_AS_MIN,USE_FAC_WHSE_SS,MAIN_FACILITY,LEAD_TIME_OVRD_DAYS_1,LEAD_TIME_OVRD_DAYS_2,LEAD_TIME_OVRD_DAYS_3,LEAD_TIME_OVRD_DAYS_4,LEAD_TIME_OVRD_DAYS_5,LEAD_TIME_OVRD_DAYS_6,LEAD_TIME_OVRD_DAYS_7,APPLY_RATIONING,OP_DAYS_1ST_RUN_1,OP_DAYS_1ST_RUN_2,OP_DAYS_1ST_RUN_3,OP_DAYS_1ST_RUN_4
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [9]:
XVM_df.columns

Index(['CS_VEND_SITE_ID', 'VENDOR_NUMBER', 'CONNECTING_VENDOR', 'VENDOR_NAME',
       'VENDOR_FACILITY', 'SHIP_TO_LOCATION_NAME', 'VENDOR_STATUS',
       'AP_VENDOR_NBR', 'AP_VENDOR_NAME', 'BUYER_ID', 'BUYER_NAME',
       'WAREHOUSE_CODE', 'FOOD_LINK_FLAG', 'ADDR_LINE1', 'ADDR_LINE2',
       'ADDR_LINE3', 'CITY', 'STATE', 'ZIP_CODE', 'COUNTRY', 'PHONE_NBR',
       'VENDOR_CONTACT_NAME', 'SHIP_TO_LOCATION_NAME', 'BILL_TO_LOCATION_NAME',
       'BROKER_NUMBER', 'BROKER_NAME', 'TRUCK_FLAG', 'RAIL_FLAG',
       'PO_TRANSMISSION_TYPE', 'PO_TRANSMISSION_VALUE'],
      dtype='object')

In [67]:
conn = psycopg2.connect(**db_params)
# Create a cursor object to execute SQL queries
cur = conn.cursor()

# Iterate over DataFrame rows for upsert
for index, row in merged_df.iterrows():
#     cur.execute("SELECT nextval('cs_vendor_id_seq')")
#     next_sequence_value = cur.fetchone()[0]
    
    
    cur.execute(
        f"INSERT INTO cspoms.XXPO_VENDOR_MASTER (CS_VENDOR_ID, VENDOR_NUMBER, VENDOR_FACILITY, CS_VEND_SITE_ID,CONNECTING_VENDOR) "
        f"VALUES (nextval('cs_vendor_id_seq'), '{row['VENDOR_NUMBER']}'::numeric, '{row['VENDOR_FACILITY']}', '{row['CS_VEND_SITE_ID']}', '{row['CONNECTING_VENDOR']}') "
        f"ON CONFLICT (VENDOR_NUMBER, VENDOR_FACILITY) DO UPDATE SET "
        f"VENDOR_NUMBER = EXCLUDED.VENDOR_NUMBER, "
        f"VENDOR_FACILITY = EXCLUDED.VENDOR_FACILITY, "
        f"CS_VEND_SITE_ID = EXCLUDED.CS_VEND_SITE_ID, "
        f"CONNECTING_VENDOR = EXCLUDED.CONNECTING_VENDOR " 
        f"RETURNING CS_VENDOR_ID;"
    )

    # Fetch the generated C1 value
    generated_c1 = cur.fetchone()[0]

    # If C1 was generated or updated, insert/update other tables
    if generated_c1:
        print('insert if')
        cur.execute(
            f"INSERT INTO XXPO_VENDOR_ORDER_POINT (CS_VENDOR_ID, MAIN_FACILITY) "
            f"VALUES ({generated_c1}, '{row['MAIN_FACILITY']}') "
            f"ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET MAIN_FACILITY = EXCLUDED.MAIN_FACILITY;"
        )
        cur.execute(
            f"INSERT INTO XXPO_VENDOR_ORDER_INFO (CS_VENDOR_ID, CONNECTING_VENDOR) "
            f"VALUES ({generated_c1}, '{row['CONNECTING_VENDOR']}') "
            f"ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET CONNECTING_VENDOR = EXCLUDED.CONNECTING_VENDOR;"
        )
        cur.execute(
            f"INSERT INTO XXPO_VENDOR_COSTING_INFO (CS_VENDOR_ID, FREIGHT_ALLOW_BUY_RATE) "
            f"VALUES ({generated_c1}, '{row['FREIGHT_ALLOW_BUY_RATE']}') "
            f"ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET FREIGHT_ALLOW_BUY_RATE = EXCLUDED.FREIGHT_ALLOW_BUY_RATE;"
        )

# Commit changes and close the connection
conn.commit()
cur.close()
conn.close()

StringDataRightTruncation: value too long for type character varying(10)


In [60]:
# Assuming 'df' is your DataFrame
columns = merged_df.columns.tolist()

# Generate the list of columns for the INSERT statement
insert_columns = ", ".join(columns)

# Generate the list of values for the VALUES statement
values = ", ".join(["%s" for _ in columns])

# Construct the dynamic SQL query
sql_query = f"INSERT INTO CSPOMS.XXPO_VENDOR_MASTER ({insert_columns}) VALUES ({values}) ON CONFLICT (VENDOR_NUMBER, VENDOR_FACILITY) DO UPDATE SET ..."

# Execute the SQL query using your preferred database library
# Replace this with your actual code to execute the query
# cursor.execute(sql_query, tuple(merged_df.values))
print(sql_query)


INSERT INTO CSPOMS.XXPO_VENDOR_MASTER (stg_seq_id, bic_vendor_facility, bic_vendor_number, bic_buyer_number, bic_crp_level, bic_status, bic_po_recommended_ind, bic_afe_type1, bic_afe_fax_number, bic_afe_contact, bic_minimum_quantity, bic_minimum_quantity_sign, bic_minimum_type, bic_bracket_quantity, bic_bracket_quantity_sign, bic_current_bkt_number, bic_current_bkt_quantity, bic_current_bkt_quantity_sign, bic_current_bkt_type, bic_maximum_quantity, bic_maximum_quantity_sign, bic_maximum_type, bic_order_interval_weeks, bic_order_interval_weeks_sign, bic_lead_time_stated_weeks, bic_lead_time_stated_wks_sign, bic_fixed_review_day1, bic_fixed_review_day2, bic_fixed_review_day3, bic_fixed_review_day4, bic_fixed_review_day5, bic_fixed_review_day6, bic_fixed_review_day7, bic_target_service_level, bic_target_service_level_sign, bic_name, bic_address_1, bic_city, bic_state, bic_zip, bic_phone, bic_contact, bic_transfer_match_type, bic_payable_vendor_nbr, bic_lead_time_weeks_type, bic_duns_remit

In [69]:
merged_df

,stg_seq_id,VENDOR_FACILITY,bic_vendor_number,bic_buyer_number,bic_crp_level,bic_status,bic_po_recommended_ind,bic_afe_type1,PO_TRANSMISSION_VALUE,PO_TRANSMISSION_TYPE,...,PICKUP_DAY_6,PICKUP_DAY_7,EOQ_CAP_DAYS,WHSE_SHIP_TO,SHIP_WITH_VENDOR,RANK_PERCENT_OVERRIDE,PO_DATE_CHANGE_RSN_FLAG,PULLBKS_AVAILABLE,HAWAII_TRANS_METHOD,TEST_NEW
0,11,252,008571,ZZ,,A,B,,,,...,,,,,,,,,,
1,12,905,008571,ZZ,,A,B,,,,...,,,,,,,,,,
2,13,251,445932,J8,,A,B,,,,...,,,,,,,,,,
3,14,252,447709,J8,,A,B,,,,...,,,,,,,,,,


In [62]:
# print(sql_query)
file_path = 'output_query.sql'

# Open
the file in write mode and write the SQL query
with open(file_path, 'w') as file:
    file.write(sql_query)

print(f"SQL query written to {file_path}")

SQL query written to output_query.sql


In [2]:
# Assuming 'df' is your DataFrame
# columns = merged_df.columns.tolist()
columns = ['CONNECTING_VENDOR','VENDOR_FACILITY','bic_vendor_number']

# Generate the list of columns for the INSERT statement
insert_columns = ", ".join(columns)

# Generate the list of values for the VALUES statement
values = ", ".join(["%s" for _ in columns])

# Generate the SET clause for the ON CONFLICT DO UPDATE statement
set_clause = ", ".join([f"{col} = EXCLUDED.{col}" for col in columns])

# Construct the dynamic SQL query
sql_query = f"INSERT INTO CSPOMS.XXPO_VENDOR_MASTER ({insert_columns}) VALUES ({values}) ON CONFLICT (VENDOR_NUMBER, VENDOR_FACILITY) DO UPDATE SET {set_clause} RETURNING CS_VENDOR_ID;"

# Open the file in write mode and write the SQL query
file_path = 'output_query.sql'
with open(file_path, 'w') as file:
    file.write(sql_query)

print(f"SQL query written to {file_path}")
sql_query

SQL query written to output_query.sql


'INSERT INTO CSPOMS.XXPO_VENDOR_MASTER (CONNECTING_VENDOR, VENDOR_FACILITY, bic_vendor_number) VALUES (%s, %s, %s) ON CONFLICT (VENDOR_NUMBER, VENDOR_FACILITY) DO UPDATE SET CONNECTING_VENDOR = EXCLUDED.CONNECTING_VENDOR, VENDOR_FACILITY = EXCLUDED.VENDOR_FACILITY, bic_vendor_number = EXCLUDED.bic_vendor_number RETURNING CS_VENDOR_ID;'

In [52]:
XVM_df['CONNECTING_VENDOR']

0    
1    
2    
3    
Name: CONNECTING_VENDOR, dtype: object